<a href="https://colab.research.google.com/github/duc-ke/study_isg_deeplearning/blob/master/%ED%85%90%EC%84%9C%ED%94%8C%EB%A1%9C%EC%9A%B0%EB%A5%BC_%ED%99%9C%EC%9A%A9%ED%95%9C_%EC%84%A0%ED%98%95%ED%9A%8C%EA%B7%80%EB%AA%A8%EB%8D%B8_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###  ◈ 텐서플로우를 활용한 선형회귀모델 구현


In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn]))

In [0]:
import tensorflow as tf # 텐서플로우 라이브러리 사용
import numpy  as np     # 파이썬 리스트로 제공되는 입력을 NumPy 배열로 바꾸기 위해 사용됩니다.
import matplotlib.pyplot as plt   # 그래프를 그리기 위해 사용됩니다.
import celluloid # gif 그래프 만드는 라이브러리
from IPython.display import HTML # HTML 화면 구성

In [0]:
# 훈련시킬 때 사용되는 파라미터
learning_rate = 0.01  # GradientDescentOptimizer에 적용할 학습률입니다. 너무 크거나 작으면 훈련결과가 좋지 않게 됩니다.
training_epochs = 1000 # 전체 반복하는 훈련 횟수입니다. 보통 1000번 훈련을 반복시킵니다.



In [0]:
train = np.loadtxt('E:/heigh_weight.csv', delimiter=',', skiprows=1)
train

array([[170.,  70.],
       [168.,  66.],
       [177.,  73.],
       [181.,  77.],
       [172.,  74.],
       [171.,  73.],
       [169.,  69.],
       [175.,  79.],
       [174.,  77.],
       [178.,  80.],
       [170.,  74.],
       [167.,  68.],
       [177.,  71.],
       [182., 276.],
       [173.,  78.],
       [171.,  72.],
       [170.,  68.],
       [179.,  79.],
       [175.,  77.],
       [177.,  81.],
       [186.,  84.],
       [166.,  73.],
       [183.,  78.],
       [168.,  69.]])

In [0]:

# 훈련 데이터(Training Data)


train_x = train[:, 0]
train_y = train[:, 1]
train_x, train_y


## 정규화 과정
mu = train_x.mean() #train_x의 평균값
sigma = train_x.std()#train_x의 표준편차()

def z_score(x):
    return (x - mu) / sigma

train_x = z_score(train_x)

num_of_samples = train_x.shape[0]# 전체 행의 개수 리턴


In [0]:
# 모델의 입력을 플레이스홀더로 정의합니다.   ---> 모델의 입력으로 플레이스 홀더를 사용합니다.
X = tf.placeholder(tf.float32)  # tf.float32 대신에 "float"를 사용하기도 합니다.
Y = tf.placeholder(tf.float32)
X,Y

(<tf.Tensor 'Placeholder_8:0' shape=<unknown> dtype=float32>,
 <tf.Tensor 'Placeholder_9:0' shape=<unknown> dtype=float32>)

In [0]:

# 모델의 출력을 결정하는 모델 파라미터는 다음 두가지 입니다.   --> 모델의 파라미터로 변수를 사용합니다.
# 변수에 값이 지정되어 있지만 나중에 초기화를 해주어야 해당 값으로 변수가 초기화됩니다.
# 여기에선 numpy.random.randn()를 사용하여 랜덤으로 숫자를 발생시켜 초기값으로 사용합니다. 초기값이 결과에 큰 영향을 주지않기 때문에 랜덤값을 사용합니다.
W = tf.Variable(np.random.randn(), name="weight")
b = tf.Variable(np.random.randn(), name="bias")

W,b

(<tf.Variable 'weight_4:0' shape=() dtype=float32_ref>,
 <tf.Variable 'bias_4:0' shape=() dtype=float32_ref>)

In [0]:

# 선형 모델(linear model)을 정의합니다. 여기에서 X는 플레이스홀더이기 때문에 모델을 계산하려면 나중에 X에 데이터를 제공해줘야 합니다.
pred = tf.add(tf.multiply(X, W), b)   # pred = X * W + b tf.add(tf.multiply(X, W), b)

# 평균 제곱 오차 MSE(Mean Squared Error)를 모델 성능 측정에 사용합니다.
# 모델에서 입력 X로부터 계산된 값 pred와 입력 X와 함께 제공된 Y가 얼마나 차이있는지 측정합니다.
#cost = tf.reduce_sum(tf.pow(pred-Y, 2))/(2*num_of_samples)
#최소자승법은 값들의 차이가 많이나면 성능저하와 오버히팅발생
#reduce_mean : 모수로 나눠주는 작업
#square : 제곱을 해주는 작업
#reduce_sum : 특정 차원을 제거하고 합계

#최소제곱법(Least square)를 모델 성능 측정에 사용합니다.
cost = 0.5* tf.reduce_mean(tf.square(pred - Y))

# 경사 하강법(Gradient descent)으로 비용(cost)를 최소화(minimize)하는 모델 파라미터 W와 b를 찾습니다. train하위 함수

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)


In [0]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
# 세션에서 실행시켜야 훈련이 시작됩니다.
with tf.Session() as sess:

    # 변수 W와 b가 앞에서 지정한 값으로 초기화됩니다.
    sess.run(tf.global_variables_initializer())

    count = 0

    # celluloid.Camera On!
    camera = celluloid.Camera(fig)  

    # Fit all training data --> fiting은 모든 훈련 데이터에 맞도록 모델의 파라미터를 수정하는 과정입니다.
    for epoch in range(training_epochs):
        count += 1
        for (x, y) in zip(train_x, train_y):   # zip에 있는 리스트들을 원소로하는 튜플을 생성하여
            plt.plot(train_x, train_y, 'o')
            plt.plot(train_x, sess.run(W) * train_x + sess.run(b), )  # 최종 결정된 모델 파라미터를 직선으로 표현합니다.
            camera.snap()  # 스샷
            sess.run(optimizer, feed_dict={X: x, Y: y})  # 모델의 입력인 플레이스홀더 X, Y에 데이터로 제공합니다. 그래야 optimizer를 실행할 수 있습니다.
    
            

        #feed_dict -> 각각의 변수에 값들 넣어줌
        c = sess.run(cost, feed_dict={X: train_x, Y:train_y})
       #if count % 20 == 0:
        print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.3f}".format(c), \
                "W=", sess.run(W), "b=", sess.run(b),"count=", count)


    print("최적화가 완료되었습니다.")
    training_cost = sess.run(cost, feed_dict={X: train_x, Y: train_y})
    print("훈련이 끝난 후 비용과 모델 파라미터입니다.  cost=", training_cost, "W=", sess.run(W), "b=", sess.run(b), '\n')

    
    
    # for loop이 종료되고나서 camera 인스턴스의 animate함수를 이용해서 이어붙이기
animation = camera.animate(interval=100, blit=True)

# jupyter에서 gif 보여주기
HTML(animation.to_html5_video())

Epoch: 0001 cost= 2859.895 W= 5.0159087 b= 17.879671 count= 1
Epoch: 0002 cost= 2030.968 W= 7.208213 b= 31.830767 count= 2
Epoch: 0003 cost= 1520.264 W= 8.932321 b= 42.77499 count= 3
Epoch: 0004 cost= 1205.528 W= 10.288207 b= 51.36041 count= 4
Epoch: 0005 cost= 1011.495 W= 11.354508 b= 58.0954 count= 5
Epoch: 0006 cost= 891.819 W= 12.193059 b= 63.378807 count= 6
Epoch: 0007 cost= 817.962 W= 12.852502 b= 67.52349 count= 7
Epoch: 0008 cost= 772.348 W= 13.371089 b= 70.77486 count= 8
Epoch: 0009 cost= 744.151 W= 13.778902 b= 73.32545 count= 9
Epoch: 0010 cost= 726.700 W= 14.099602 b= 75.326294 count= 10
Epoch: 0011 cost= 715.884 W= 14.35179 b= 76.89587 count= 11
Epoch: 0012 cost= 709.167 W= 14.550106 b= 78.127174 count= 12
Epoch: 0013 cost= 704.986 W= 14.706056 b= 79.09308 count= 13
Epoch: 0014 cost= 702.376 W= 14.828687 b= 79.85079 count= 14
Epoch: 0015 cost= 700.741 W= 14.925116 b= 80.44519 count= 15
Epoch: 0016 cost= 699.711 W= 15.000943 b= 80.911476 count= 16
Epoch: 0017 cost= 699.060 

Epoch: 0135 cost= 697.848 W= 15.279977 b= 82.60853 count= 135
Epoch: 0136 cost= 697.848 W= 15.279977 b= 82.60853 count= 136
Epoch: 0137 cost= 697.848 W= 15.279977 b= 82.60853 count= 137
Epoch: 0138 cost= 697.848 W= 15.279977 b= 82.60853 count= 138
Epoch: 0139 cost= 697.848 W= 15.279977 b= 82.60853 count= 139
Epoch: 0140 cost= 697.848 W= 15.279977 b= 82.60853 count= 140
Epoch: 0141 cost= 697.848 W= 15.279977 b= 82.60853 count= 141
Epoch: 0142 cost= 697.848 W= 15.279977 b= 82.60853 count= 142
Epoch: 0143 cost= 697.848 W= 15.279977 b= 82.60853 count= 143
Epoch: 0144 cost= 697.848 W= 15.279977 b= 82.60853 count= 144
Epoch: 0145 cost= 697.848 W= 15.279977 b= 82.60853 count= 145
Epoch: 0146 cost= 697.848 W= 15.279977 b= 82.60853 count= 146
Epoch: 0147 cost= 697.848 W= 15.279977 b= 82.60853 count= 147
Epoch: 0148 cost= 697.848 W= 15.279977 b= 82.60853 count= 148
Epoch: 0149 cost= 697.848 W= 15.279977 b= 82.60853 count= 149
Epoch: 0150 cost= 697.848 W= 15.279977 b= 82.60853 count= 150
Epoch: 0

Epoch: 0268 cost= 697.848 W= 15.279977 b= 82.60853 count= 268
Epoch: 0269 cost= 697.848 W= 15.279977 b= 82.60853 count= 269
Epoch: 0270 cost= 697.848 W= 15.279977 b= 82.60853 count= 270
Epoch: 0271 cost= 697.848 W= 15.279977 b= 82.60853 count= 271
Epoch: 0272 cost= 697.848 W= 15.279977 b= 82.60853 count= 272
Epoch: 0273 cost= 697.848 W= 15.279977 b= 82.60853 count= 273
Epoch: 0274 cost= 697.848 W= 15.279977 b= 82.60853 count= 274
Epoch: 0275 cost= 697.848 W= 15.279977 b= 82.60853 count= 275
Epoch: 0276 cost= 697.848 W= 15.279977 b= 82.60853 count= 276
Epoch: 0277 cost= 697.848 W= 15.279977 b= 82.60853 count= 277
Epoch: 0278 cost= 697.848 W= 15.279977 b= 82.60853 count= 278
Epoch: 0279 cost= 697.848 W= 15.279977 b= 82.60853 count= 279
Epoch: 0280 cost= 697.848 W= 15.279977 b= 82.60853 count= 280
Epoch: 0281 cost= 697.848 W= 15.279977 b= 82.60853 count= 281
Epoch: 0282 cost= 697.848 W= 15.279977 b= 82.60853 count= 282
Epoch: 0283 cost= 697.848 W= 15.279977 b= 82.60853 count= 283
Epoch: 0

Epoch: 0401 cost= 697.848 W= 15.279977 b= 82.60853 count= 401
Epoch: 0402 cost= 697.848 W= 15.279977 b= 82.60853 count= 402
Epoch: 0403 cost= 697.848 W= 15.279977 b= 82.60853 count= 403
Epoch: 0404 cost= 697.848 W= 15.279977 b= 82.60853 count= 404
Epoch: 0405 cost= 697.848 W= 15.279977 b= 82.60853 count= 405
Epoch: 0406 cost= 697.848 W= 15.279977 b= 82.60853 count= 406
Epoch: 0407 cost= 697.848 W= 15.279977 b= 82.60853 count= 407
Epoch: 0408 cost= 697.848 W= 15.279977 b= 82.60853 count= 408
Epoch: 0409 cost= 697.848 W= 15.279977 b= 82.60853 count= 409
Epoch: 0410 cost= 697.848 W= 15.279977 b= 82.60853 count= 410
Epoch: 0411 cost= 697.848 W= 15.279977 b= 82.60853 count= 411
Epoch: 0412 cost= 697.848 W= 15.279977 b= 82.60853 count= 412
Epoch: 0413 cost= 697.848 W= 15.279977 b= 82.60853 count= 413
Epoch: 0414 cost= 697.848 W= 15.279977 b= 82.60853 count= 414
Epoch: 0415 cost= 697.848 W= 15.279977 b= 82.60853 count= 415
Epoch: 0416 cost= 697.848 W= 15.279977 b= 82.60853 count= 416
Epoch: 0

Epoch: 0534 cost= 697.848 W= 15.279977 b= 82.60853 count= 534
Epoch: 0535 cost= 697.848 W= 15.279977 b= 82.60853 count= 535
Epoch: 0536 cost= 697.848 W= 15.279977 b= 82.60853 count= 536
Epoch: 0537 cost= 697.848 W= 15.279977 b= 82.60853 count= 537
Epoch: 0538 cost= 697.848 W= 15.279977 b= 82.60853 count= 538
Epoch: 0539 cost= 697.848 W= 15.279977 b= 82.60853 count= 539
Epoch: 0540 cost= 697.848 W= 15.279977 b= 82.60853 count= 540
Epoch: 0541 cost= 697.848 W= 15.279977 b= 82.60853 count= 541
Epoch: 0542 cost= 697.848 W= 15.279977 b= 82.60853 count= 542
Epoch: 0543 cost= 697.848 W= 15.279977 b= 82.60853 count= 543
Epoch: 0544 cost= 697.848 W= 15.279977 b= 82.60853 count= 544
Epoch: 0545 cost= 697.848 W= 15.279977 b= 82.60853 count= 545
Epoch: 0546 cost= 697.848 W= 15.279977 b= 82.60853 count= 546
Epoch: 0547 cost= 697.848 W= 15.279977 b= 82.60853 count= 547
Epoch: 0548 cost= 697.848 W= 15.279977 b= 82.60853 count= 548
Epoch: 0549 cost= 697.848 W= 15.279977 b= 82.60853 count= 549
Epoch: 0